This notebook verifies the pseudo sequences from 

    ../../data/intermediate_data/pseudosequence_2016_all_X.dat

for HLA-II pairs with the corresponding pseudo sequences reconstructed based on six 

    prot.alfas 

files. 

A summary of the procedures and the findings of this notebook is put in the file

    t3_summary.md

In [2]:
import numpy as np
import pandas as pd

from collections import defaultdict
from collections import Counter

import os
import re

In [4]:
HLA_2_pseudo = pd.read_csv("../../data/intermediate_data/pseudosequence_2016_all_X.dat", 
                           sep = "\t", header = None)

HLA_2_pseudo.shape

(5636, 2)

In [6]:
HLA_2_pseudo.columns = ["HLA", "seq"]
HLA_2_pseudo[:6]

,HLA,seq
0,DRB1_0101,QEFFIASGAAVDAIMWLFLECYDLQRATYHVGFT
1,DRB1_0102,QEFFIASGAAVDAIMWLFLECYDLQRATYHAVFT
2,DRB1_0103,QEFFIASGAAVDAIMWLFLECYDIDEATYHVGFT
3,DRB1_0104,QEFFIASGAAVDAIMWLFLECYDLQRANYHVVFT
4,DRB1_0105,QEFFIASGAAVDAIMWLFLECYDLQRATYHVGFT
5,DRB1_0106,QEFFIASGAAVDAIMWLFLECYDLQAATYHVVFT


In [7]:
Counter([len(seq) for seq in HLA_2_pseudo.seq.tolist()])

Counter({34: 5636})

In [8]:
HLA_counter = Counter(HLA_2_pseudo.HLA)

In [9]:
for hla in list(HLA_counter.keys()):
    if HLA_counter[hla] > 1:
        print(hla)

HLA-DPA10103-DPB10601


In [10]:
# this one is the only duplicate in HLA and with difference sequences
HLA_2_pseudo_dup = HLA_2_pseudo[HLA_2_pseudo.HLA == 'HLA-DPA10103-DPB10601']
HLA_2_pseudo_dup

,HLA,seq
675,HLA-DPA10103-DPB10601,YAFFMFSGGAILNTLYLQFEYFDLEEVRMHLDVT
676,HLA-DPA10103-DPB10601,YAFFQFSGGAILNTLYLQFEYFDLEEVRMHLDVT


In [12]:
HLA_2_set_list = list(set(HLA_2_pseudo.HLA))
len(HLA_2_set_list)

5635

In [32]:
#HLA_2_set_list[-100:]

In [13]:
# position resources: 
# paper: NetMHCIIpan-3.0, a common pan-specific MHC class II prediction method 
#        including all three human MHC class II isotypes, HLA-DR, HLA-DP and HLA-DQ
pos_alpha = [9, 11, 22, 24, 31, 52, 53, 58, 59, 61, 65, 66, 68, 72, 73]
pos_beta = [9, 11, 13, 26, 28, 30, 47, 57, 67, 70, 71, 74, 77, 78, 81, 85, 86, 89, 90]

Load hla_features v1 names

In [3]:
cmd2 = "cut -f1,2 -d ' ' ../../data/intermediate_data/DeWitt_2018/HLA_features.txt > ../../data/intermediate_data/DeWitt_2018/HLA_features_row_names.txt"
os.system(cmd2)

0

In [4]:
HLA_features_row_names = pd.read_csv("../../data/intermediate_data/DeWitt_2018/HLA_features_row_names.txt", 
                                     sep = " ", header = None)
HLA_features_row_names.columns = ["feature", "hla"]
HLA_features_row_names[:6]

,feature,hla
0,feature:,HLA-A*33:01
1,feature:,HLA-A*33:03
2,feature:,HLA-A*26:01
3,feature:,HLA-A*31:01
4,feature:,HLA-A*11:01
5,feature:,HLA-A*23:01


In [17]:
HLA_features_row_names.shape

(173, 2)

In [18]:
HLA_v2_features_row_names = pd.read_csv("../../data/intermediate_data/DeWitt_2018/HLA_v2_features_row_names.txt", sep = " ", header = None)
HLA_v2_features_row_names.columns = ["feature", "hla"]
HLA_v2_features_row_names[:6]

,feature,hla
0,feature:,HLA-DPAB*02:01_04:01
1,feature:,HLA-DQAB*05:05_06:04
2,feature:,HLA-B*08:01
3,feature:,HLA-A*24:02
4,feature:,HLA-A*24:03
5,feature:,HLA-B*38:02


In [19]:
HLA_v2_features_row_names.shape

(215, 2)

In [188]:
Counter([hla[:7] for hla in HLA_v2_features_row_names.hla.tolist()])

Counter({'HLA-A*0': 6,
         'HLA-A*1': 1,
         'HLA-A*2': 7,
         'HLA-A*3': 7,
         'HLA-A*6': 3,
         'HLA-B*0': 3,
         'HLA-B*1': 9,
         'HLA-B*2': 1,
         'HLA-B*3': 9,
         'HLA-B*4': 10,
         'HLA-B*5': 8,
         'HLA-C*0': 14,
         'HLA-C*1': 7,
         'HLA-DPA': 25,
         'HLA-DQA': 67,
         'HLA-DRB': 33,
         'HLA-DRD': 5})

In [71]:
# list of allele names in v2 which are neither HLA-I nor HLA-DR
list_extra_v2 = [hla for hla in HLA_v2_features_row_names.hla.tolist() if hla[:7] in ["HLA-DPA", "HLA-DQA", "HLA-DRD"]]

len(list_extra_v2)

97

In [58]:
list_extra_v2_DRB = [hla for hla in HLA_v2_features_row_names.hla.tolist() if hla[:7] == 'HLA-DRB']

len(list_extra_v2_DRB)

33

Check whether the 97 combinations in 

    ../../data/intermediate_data/DeWitt_2018/HLA_v2_features 

have the corresponding pseudo sequence from 

    ../../data/intermediate_data/DeWitt_2018/pseudosequence_2016_all_X.dat

as compared from 

    DPA1_prot.alfas
    DPB1_prot.alfas
    DQA1_prot.alfas
    DQB1_prot.alfas
    DRA_prot.alfas
    DRB_prot.alfas 

Check the same thing for the 33 HLA-DRB. 

0. First check whether items related to HLA-II alleles in v2 have corresponding part in v1;

1. Then verify the sequence alignment.

In [36]:
# 1. First check whether items related to HLA-II alleles in v2 have corresponding part in v1
# get the set of HLA-II alleles in v1
HLA_v1_features_II = [hla for hla in HLA_features_row_names.hla.tolist() if hla[:5] == "HLA-D"]
len(HLA_v1_features_II)

87

In [43]:
HLA_v1_features_II[:6]

['HLA-DRB1*12:01',
 'HLA-DRB1*12:02',
 'HLA-DRB1*11:01',
 'HLA-DRB1*11:03',
 'HLA-DRB1*11:02',
 'HLA-DRB1*11:04']

In [34]:
list_extra_v2[:6]

['HLA-DPAB*02:01_04:01',
 'HLA-DQAB*05:05_06:04',
 'HLA-DPAB*02:01_04:02',
 'HLA-DRDQ*10:01_01:05_05:01',
 'HLA-DQAB*05:05_03:01',
 'HLA-DQAB*05:05_03:02']

In [50]:
# check whether all components in list_extra_v2 and list_extra_v2_DRB exist in v1 alleles
components_PQR = set()

for item in list_extra_v2:
    if item[4:8] == "DQAB":
        item_1 = "HLA-DQA1" + "*" + item[9:].split("_")[0]
        item_2 = "HLA-DQB1" + "*" + item[9:].split("_")[1]
        components_PQR.add(item_1)
        components_PQR.add(item_2)
    elif item[4:8] == "DPAB":
        item_1 = "HLA-DPA1" + "*" + item[9:].split("_")[0]
        item_2 = "HLA-DPB1" + "*" + item[9:].split("_")[1]
        components_PQR.add(item_1)
        components_PQR.add(item_2)
    elif item[4:8] == "DRDQ":
        item_1 = "HLA-DRB1" + "*" + item[9:].split("_")[0]
        item_2 = "HLA-DQA1" + "*" + item[9:].split("_")[1]
        item_3 = "HLA-DQB1" + "*" + item[9:].split("_")[2]
        components_PQR.add(item_1)
        components_PQR.add(item_2)
        components_PQR.add(item_3)
    else:
        print("error found, first eight letters exception")
        print(item)
        break

for item in list_extra_v2_DRB:
    if item[4:8] == "DRB1":
        item_1 = "HLA-DRB1" + "*" + item[9:]
        components_PQR.add(item_1)
    else:
        print("error found, first eight letters exception")
        print(item)
        break    

In [65]:
# here we verified that all the components of items related to HLA-II alleles in v2 exist in v1
len(components_PQR)
# 78
len(components_PQR - set(HLA_v1_features_II))

0

 verify the sequence alignment

In [67]:
# create a set of the items related to HLA-II alleles that we care about
list_extra_v2_all = [hla for hla in HLA_v2_features_row_names.hla.tolist() if hla[:7] in ["HLA-DPA", "HLA-DQA", "HLA-DRD", "HLA-DRB"]]

set(list_extra_v2_all) == set(list_extra_v2).union(set(list_extra_v2_DRB))

list_extra_v2_5DRDQ = [item for item in list_extra_v2_all if len(item.split("_")) > 2]
list_extra_v2_5DRDQ

list_extra_v2_5DRDQ_DRB = ["HLA-DRB1*" + item[9:].split("_")[0] for item in list_extra_v2_5DRDQ]
list_extra_v2_5DRDQ_DRB

list_extra_v2_5DRDQ_DQAB = ["HLA-DQAB*" + "_".join(item[9:].split("_")[1:]) for item in list_extra_v2_5DRDQ]
list_extra_v2_5DRDQ_DQAB

list_HLA_II_complete = list(set(list_extra_v2_all + list_extra_v2_5DRDQ_DRB + list_extra_v2_5DRDQ_DQAB) - set(list_extra_v2_5DRDQ))
len(list_HLA_II_complete)
# 135

list_HLA_II_complete[:6]
# now the list list_HLA_II_complete contains the 135 items related to HLA-II that we care about

In [96]:
Counter([item[:8] for item in list_HLA_II_complete])

Counter({'HLA-DPAB': 25, 'HLA-DQAB': 72, 'HLA-DRB1': 38})

In [203]:
# check whether those rows with hla name starting from 'HLA' are all either DP or DQ pairs
# from the results we can see that rows correspond to DRBs do not start with 'HLA'
HLA_2_pseudo['first_three'] = [hla[:3] for hla in HLA_2_pseudo.HLA.tolist()]
HLA_2_pseudo_check_sub = HLA_2_pseudo[HLA_2_pseudo.first_three == 'HLA']
HLA_2_pseudo_check_sub_begin = \
 ["-".join([hla.split("-")[0], hla.split("-")[1][:4], hla.split("-")[2][:4]]) for hla in HLA_2_pseudo_check_sub.HLA.tolist()]
Counter(HLA_2_pseudo_check_sub_begin)

Counter({'HLA-DPA1-DPB1': 2049, 'HLA-DQA1-DQB1': 2912})

In [206]:
HLA_2_pseudo[:6]

,HLA,seq,first_three
0,DRB1_0101,QEFFIASGAAVDAIMWLFLECYDLQRATYHVGFT,DRB
1,DRB1_0102,QEFFIASGAAVDAIMWLFLECYDLQRATYHAVFT,DRB
2,DRB1_0103,QEFFIASGAAVDAIMWLFLECYDIDEATYHVGFT,DRB
3,DRB1_0104,QEFFIASGAAVDAIMWLFLECYDLQRANYHVVFT,DRB
4,DRB1_0105,QEFFIASGAAVDAIMWLFLECYDLQRATYHVGFT,DRB
5,DRB1_0106,QEFFIASGAAVDAIMWLFLECYDLQAATYHVVFT,DRB


In [207]:
# see the format of the hla names for DRB
HLA_2_pseudo_sub_DRB = HLA_2_pseudo[HLA_2_pseudo.first_three == 'DRB']
#Counter(HLA_2_pseudo_sub_DRB.HLA)

In [543]:
#list_HLA_II_complete

In [304]:
# this first dictionary holds the corresponding two alleles of each HLA-II pair
sep_hla_II_dict = defaultdict(list)
# this second dictionary holds the translate of one HLA-II pair to the pseudo sequence in "../data/pseudosequence_2016_all_X.dat"
trans_hla_II_dict = defaultdict(str)

# separate the HLA-II pairs into two alleles each
# translate them into the names in file "../data/pseudosequence_2016_all_X.dat"
for item in list_HLA_II_complete:
    if item[:8] == "HLA-DQAB":
        item_1 = "DQA1" + "*" + item[9:].split("_")[0]
        item_2 = "DQB1" + "*" + item[9:].split("_")[1]
        sep_hla_II_dict[item] = [item_1, item_2]
        trans_hla_II_dict[item] = "HLA-" + item_1.replace("*", "").replace(":", "") + "-" + item_2.replace("*", "").replace(":", "")
    elif item[:8] == "HLA-DPAB":
        item_1 = "DPA1" + "*" + item[9:].split("_")[0]
        item_2 = "DPB1" + "*" + item[9:].split("_")[1]
        sep_hla_II_dict[item] = [item_1, item_2]
        trans_hla_II_dict[item] = "HLA-" + item_1.replace("*", "").replace(":", "") + "-" + item_2.replace("*", "").replace(":", "")
    elif item[:8] == "HLA-DRB1":
        item_1 = "DRA"
        item_2 = "DRB1" + "*" + item[9:]
        sep_hla_II_dict[item] = [item_1, item_2]
        trans_hla_II_dict[item] = 'DRB1_' + item[9:].replace(":", "")
    else:
        print("error found, first eight letters exception")
        print(item)
        break

In [305]:
len(sep_hla_II_dict)

135

In [306]:
len(trans_hla_II_dict)

135

In [126]:
#trans_hla2_dict

build pseudo sequence dictionary

In [307]:
# the duplicate does not show up in v2 as one combination, just assign the last seq to it when 
# constructing dictionary
"HLA-DPAB*01:03_06:01" in list_HLA_II_complete

False

In [308]:
HLA_2_pseudo_dict = defaultdict(str)

for hla, seq in zip(HLA_2_pseudo.HLA.tolist(), HLA_2_pseudo.seq.tolist()):
    HLA_2_pseudo_dict[hla] = seq

len(HLA_2_pseudo_dict)

5635

 load sequence info files

Load DRA sequence

In [309]:
DRA_prot = pd.read_csv("../../data/intermediate_data/DeWitt_2018/HLA_TCR_contact/DRA_prot.alfas", 
                       sep = " ", header = None)
DRA_prot.shape

(14, 1)

In [310]:
name_ind = list(range(int(DRA_prot.shape[0]/2)))
names = DRA_prot.loc[[2 * ind for ind in name_ind]]
names = names.iloc[:, 0].tolist()
names = [name.replace(">", "") for name in names]
seqs = DRA_prot.loc[[2 * ind + 1 for ind in name_ind]]
seqs = seqs.iloc[:, 0].tolist()

DRA_seqs = pd.DataFrame(list(zip(names, seqs)), 
                            columns =['name', 'seq']) 
DRA_seqs.shape

(7, 2)

In [340]:
DRA_seqs

,name,seq
0,DRA*01:01:01:01,HVIIQ-AEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFG...
1,DRA*01:01:01:02,HVIIQ-AEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFG...
2,DRA*01:01:01:03,HVIIQ-AEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFG...
3,DRA*01:01:02,HVIIQ-AEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFG...
4,DRA*01:02:01,HVIIQ-AEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFG...
5,DRA*01:02:02,HVIIQ-AEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFG...
6,DRA*01:02:03,HVIIQ-AEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFG...


In [302]:
# 31 (0-indexed) <--> 35 (1-indexed)
DRA_seq_unique = list(set(DRA_seqs.seq.tolist()))[0]
DRA_seq_unique

'HVIIQ-AEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFGRFASFEAQGALANIAVDKANLEIMTKRSN'

Load DRB sequences

In [311]:
DRB_prot = pd.read_csv("../../data/intermediate_data/DeWitt_2018/HLA_TCR_contact/DRB_prot.alfas", 
                       sep = " ", header = None)
DRB_prot.shape

(4476, 1)

In [312]:
name_ind = list(range(int(DRB_prot.shape[0]/2)))
names = DRB_prot.loc[[2 * ind for ind in name_ind]]
names = names.iloc[:, 0].tolist()
names = [name.replace(">", "") for name in names]
seqs = DRB_prot.loc[[2 * ind + 1 for ind in name_ind]]
seqs = seqs.iloc[:, 0].tolist()

DRB_seqs = pd.DataFrame(list(zip(names, seqs)), 
                            columns =['name', 'seq']) 
DRB_seqs.shape

(2238, 2)

In [154]:
DRB_seqs[:6]

,name,seq
0,DRB1*01:01:01,FLWQLKFECHFFNGTERVRLLERCIYNQEESVRFDSDVGEYRAVTE...
1,DRB1*01:01:02,FLWQLKFECHFFNGTERVRLLERCIYNQEESVRFDSDVGEYRAVTE...
2,DRB1*01:01:03,FLWQLKFECHFFNGTERVRLLERCIYNQEESVRFDSDVGEYRAVTE...
3,DRB1*01:01:04,FLWQLKFECHFFNGTERVRLLERCIYNQEESVRFDSDVGEYRAVTE...
4,DRB1*01:01:05,FLWQLKFECHFFNGTERVRLLERCIYNQEESVRFDSDVGEYRAVTE...
5,DRB1*01:01:06,FLWQLKFECHFFNGTERVRLLERCIYNQEESVRFDSDVGEYRAVTE...


In [341]:
DRB_seqs.nunique()

name     2238
seq      1641
short    1709
dtype: int64

In [313]:
DRB_seqs['short'] = [":".join(name.split(":")[:2]) for name in DRB_seqs.name.tolist()]

In [314]:
DRB_seq_dict = defaultdict(set)
for short, seq in zip(DRB_seqs.short.tolist(), DRB_seqs.seq.tolist()):
    DRB_seq_dict[short].add(seq)

len(DRB_seq_dict)

1709

In [342]:
Counter([len(value) for value in DRB_seq_dict.values()])

Counter({1: 1697, 2: 11, 3: 1})

Load DPA1 sequences

In [315]:
DPA1_prot = pd.read_csv("../../data/intermediate_data/DeWitt_2018/HLA_TCR_contact/DPA1_prot.alfas", 
                        sep = " ", header = None)
DPA1_prot.shape

(106, 1)

In [316]:
name_ind = list(range(int(DPA1_prot.shape[0]/2)))
names = DPA1_prot.loc[[2 * ind for ind in name_ind]]
names = names.iloc[:, 0].tolist()
names = [name.replace(">", "") for name in names]
seqs = DPA1_prot.loc[[2 * ind + 1 for ind in name_ind]]
seqs = seqs.iloc[:, 0].tolist()

DPA1_seqs = pd.DataFrame(list(zip(names, seqs)), 
                            columns =['name', 'seq']) 
DPA1_seqs.shape

(53, 2)

In [317]:
DPA1_seqs['short'] = [":".join(name.split(":")[:2]) for name in DPA1_seqs.name.tolist()]

In [318]:
DPA1_seq_dict = defaultdict(set)
for short, seq in zip(DPA1_seqs.short.tolist(), DPA1_seqs.seq.tolist()):
    DPA1_seq_dict[short].add(seq)

len(DPA1_seq_dict)

24

Load DPB1 sequences

In [319]:
DPB1_prot = pd.read_csv("../../data/intermediate_data/DeWitt_2018/HLA_TCR_contact/DPB1_prot.alfas", 
                        sep = " ", header = None)
DPB1_prot.shape

(1746, 1)

In [320]:
name_ind = list(range(int(DPB1_prot.shape[0]/2)))
names = DPB1_prot.loc[[2 * ind for ind in name_ind]]
names = names.iloc[:, 0].tolist()
names = [name.replace(">", "") for name in names]
seqs = DPB1_prot.loc[[2 * ind + 1 for ind in name_ind]]
seqs = seqs.iloc[:, 0].tolist()

DPB1_seqs = pd.DataFrame(list(zip(names, seqs)), 
                            columns =['name', 'seq']) 
DPB1_seqs.shape

(873, 2)

In [321]:
DPB1_seqs['short'] = [":".join(name.split(":")[:2]) for name in DPB1_seqs.name.tolist()]

In [322]:
DPB1_seq_dict = defaultdict(set)
for short, seq in zip(DPB1_seqs.short.tolist(), DPB1_seqs.seq.tolist()):
    DPB1_seq_dict[short].add(seq)

len(DPB1_seq_dict)

641

Load DQA1 sequences

In [323]:
DQA1_prot = pd.read_csv("../../data/intermediate_data/DeWitt_2018/HLA_TCR_contact/DQA1_prot.alfas", 
                        sep = " ", header = None)
DQA1_prot.shape

(182, 1)

In [324]:
name_ind = list(range(int(DQA1_prot.shape[0]/2)))
names = DQA1_prot.loc[[2 * ind for ind in name_ind]]
names = names.iloc[:, 0].tolist()
names = [name.replace(">", "") for name in names]
seqs = DQA1_prot.loc[[2 * ind + 1 for ind in name_ind]]
seqs = seqs.iloc[:, 0].tolist()

DQA1_seqs = pd.DataFrame(list(zip(names, seqs)), 
                            columns =['name', 'seq']) 
DQA1_seqs.shape

(91, 2)

In [325]:
DQA1_seqs['short'] = [":".join(name.split(":")[:2]) for name in DQA1_seqs.name.tolist()]

In [326]:
DQA1_seq_dict = defaultdict(set)
for short, seq in zip(DQA1_seqs.short.tolist(), DQA1_seqs.seq.tolist()):
    DQA1_seq_dict[short].add(seq)

len(DQA1_seq_dict)

35

Load DQB1 sequences

In [327]:
DQB1_prot = pd.read_csv("../../data/intermediate_data/DeWitt_2018/HLA_TCR_contact/DQB1_prot.alfas", 
                        sep = " ", header = None)
DQB1_prot.shape

(2090, 1)

In [328]:
name_ind = list(range(int(DQB1_prot.shape[0]/2)))
names = DQB1_prot.loc[[2 * ind for ind in name_ind]]
names = names.iloc[:, 0].tolist()
names = [name.replace(">", "") for name in names]
seqs = DQB1_prot.loc[[2 * ind + 1 for ind in name_ind]]
seqs = seqs.iloc[:, 0].tolist()

DQB1_seqs = pd.DataFrame(list(zip(names, seqs)), 
                            columns =['name', 'seq']) 
DQB1_seqs.shape

(1045, 2)

In [329]:
DQB1_seqs['short'] = [":".join(name.split(":")[:2]) for name in DQB1_seqs.name.tolist()]

In [330]:
DQB1_seq_dict = defaultdict(set)
for short, seq in zip(DQB1_seqs.short.tolist(), DQB1_seqs.seq.tolist()):
    DQB1_seq_dict[short].add(seq)

len(DQB1_seq_dict)

748

First, try finding the alignment starting position based on matching DRA sequence from 

    DRA_prot.alfas 

and that from NetMHCIIpan-3.0 paper.

In [221]:
DRA_seq_unique

'HVIIQ-AEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFGRFASFEAQGALANIAVDKANLEIMTKRSN'

In [ ]:
'DMAKKETVWRLEEFGRFASFEAQGALANIAVDKANLEIMTKRSNYTPITNVPPEVTVLTN'

In [225]:
# 31 (0-indexed) <--> 35 (1-indexed)
DRA_seq_unique[31:]

'DMAKKETVWRLEEFGRFASFEAQGALANIAVDKANLEIMTKRSN'

In [227]:
"".join([DRA_seq_unique[ind-4] for ind in pos_alpha])

'-EFFIASGAAVDAIM'

In [231]:
'QEFFIASGAAVDAIMWLFLECYDLQRATYHVGFT'[:15] == 'QEFFIASGAAVDAIM'

True

try finding the alignment starting position based on matching DRB sequence from 

    DRB_prot.alfas 

and that from 

    ../data/pseudosequence_2016_all_X.dat

In [233]:
sep_hla_II_dict['HLA-DRB1*01:01']

['DRA', 'DRB1*01:01']

In [244]:
test_DRB_seq = list(DRB_seq_dict['DRB1*01:01'])[0]

In [245]:
test_DRB_seq

'FLWQLKFECHFFNGTERVRLLERCIYNQEESVRFDSDVGEYRAVTELGRPDAEYWNSQKDLLEQRRAAVDTYCRHNYGVGESFTV'

In [240]:
pos_beta

[9, 11, 13, 26, 28, 30, 47, 57, 67, 70, 71, 74, 77, 78, 81, 85, 86, 89, 90]

In [241]:
trans_hla_II_dict['HLA-DRB1*01:01']

'DRB1_0101'

In [242]:
'QEFFIASGAAVDAIMWLFLECYDLQRATYHVGFT'[15:]

'WLFLECYDLQRATYHVGFT'

In [247]:
# 2 (0-indexed) <--> 9 (1-indexed)
''.join([test_DRB_seq[ind-7] for ind in pos_beta]) == 'QEFFIASGAAVDAIMWLFLECYDLQRATYHVGFT'[15:]


True

In [334]:
Counter([len(value) for value in list(DRB_seq_dict.values())])

Counter({1: 1697, 2: 11, 3: 1})

In [336]:
#[value for value in list(DRB_seq_dict.values()) if len(value) > 1]

trying finding the alignment position for DPA

based on 

    DPA_prot.alfas 

and 

    ../data/pseudosequence_2016_all_X.dat

In [256]:
sep_hla_II_dict['HLA-DPAB*01:03_01:01']

['DPA1*01:03', 'DPB1*01:01']

In [257]:
trans_hla_II_dict['HLA-DPAB*01:03_01:01']

'HLA-DPA10103-DPB10101'

In [265]:
HLA_2_pseudo_dict['HLA-DPA10103-DPB10101'][:15] == 'YAFFMFSGGAILNTL'

True

In [263]:
DPA1_seq_dict['DPA1*01:03']

{'HVSTY-AAFVQTHRPTGEFMFEFDEDEMFYVDLDKKETVWHLEEFGQAFSFEAQGGLANIAILNNNLNTLIQRSN',
 'XXXXX-XAFVQTHRPTGEFMFEFDEDEMFYVDLDKKETVWHLEEFGQAFSFEAQGGLANIAILNNNLNTLIQRSN'}

In [264]:
"".join(['HVSTY-AAFVQTHRPTGEFMFEFDEDEMFYVDLDKKETVWHLEEFGQAFSFEAQGGLANIAILNNNLNTLIQRSN'[ind-4] for ind in pos_alpha]) 

'-AFFMFSGGAILNTL'

In [266]:
"".join(['XXXXX-XAFVQTHRPTGEFMFEFDEDEMFYVDLDKKETVWHLEEFGQAFSFEAQGGLANIAILNNNLNTLIQRSN'[ind-4] for ind in pos_alpha])

'-AFFMFSGGAILNTL'

In [332]:
Counter([len(value) for value in list(DPA1_seq_dict.values())])

Counter({1: 21, 2: 3})

In [333]:
[value for value in list(DPA1_seq_dict.values()) if len(value) > 1]

[{'HVSTY-AAFVQTHRPTGEFMFEFDEDEMFYVDLDKKETVWHLEEFGQAFSFEAQGGLANIAILNNNLNTLIQRSN',
  'XXXXX-XAFVQTHRPTGEFMFEFDEDEMFYVDLDKKETVWHLEEFGQAFSFEAQGGLANIAILNNNLNTLIQRSN'},
 {'HVSTY-AAFVQTHRPTGEFMFEFDEDEQFYVDLDKKETVWHLEEFGRAFSFEAQGGLANIAILNNNLNTLIQRSN',
  'XXXXX-XAFVQTHRPTGEFMFEFDEDEQFYVDLDKKETVWHLEEFGRAFSFEAQGGLANIAILNNNLNTLIQRSN'},
 {'HVSTY-AMFVQTHRPTGEFMFEFDEDEQFYVDLDKKETVWHLEEFGRAFSFEAQGGLANIAILNNNLNTLIQRSN',
  'XVSTY-AMFVQTHRPTGEFMFEFDEDEQFYVDLDKKETVWHLEEFGRAFSFEAQGGLANIAILNNNLNTLIQRSN'}]

try finding alignment of DPB, based on 

    DPB_prot.alfas

and

    ../data/pseudosequence_2016_all_X.dat

In [268]:
DPB1_seq_dict['DPB1*01:01']

{'YVYQGRQECYAFN--GTQRFLERYIYNREEYARFDSDVGEFRAVTELGRPAAEYWNSQKDILEEKRAVPDRVCRHNYELDEAVTL'}

In [269]:
"".join(['YVYQGRQECYAFN--GTQRFLERYIYNREEYARFDSDVGEFRAVTELGRPAAEYWNSQKDILEEKRAVPDRVCRHNYELDEAVTL'[ind-7] for ind in pos_beta]) 

'YGQFEYFAIEKVRVHLDVT'

In [271]:
HLA_2_pseudo_dict['HLA-DPA10103-DPB10101'][15:] == 'YGQFEYFAIEKVRVHLDVT'

True

In [278]:
Counter([len(value) for value in list(DPB1_seq_dict.values())])

Counter({1: 640, 2: 1})

In [274]:
[value for value in list(DPB1_seq_dict.values()) if len(value) == 2]

[{'XVYQLRQECYAFN--GTQRFLERYIYNREEYARFDSDVGEFRAVTELGRPAAEYWNSQKDILEEKRAVPDRVCRHNYELDEAVTL',
  'YVYQLRQECYAFN--GTQRFLERYIYNREEYARFDSDVGEFRAVTELGRPAAEYWNSQKDILEEKRAVPDRVCRHNYELDEAVTL'}]

In [275]:
"".join(['XVYQLRQECYAFN--GTQRFLERYIYNREEYARFDSDVGEFRAVTELGRPAAEYWNSQKDILEEKRAVPDRVCRHNYELDEAVTL'[ind-7] for ind in pos_beta]) 

'YLQFEYFAIEKVRVHLDVT'

In [277]:
"".join(['YVYQLRQECYAFN--GTQRFLERYIYNREEYARFDSDVGEFRAVTELGRPAAEYWNSQKDILEEKRAVPDRVCRHNYELDEAVTL'[ind-7] for ind in pos_beta]) == 'YLQFEYFAIEKVRVHLDVT'

True

try finding alignment of DQA, based on 

    DQA_prot.alfas 

and

    ../data/pseudosequence_2016_all_X.dat

In [279]:
sep_hla_II_dict['HLA-DQAB*01:05_05:01']

['DQA1*01:05', 'DQB1*05:01']

In [280]:
trans_hla_II_dict['HLA-DQAB*01:05_05:01']

'HLA-DQA10105-DQB10501'

In [287]:
HLA_2_pseudo_dict['HLA-DQA10105-DQB10501'][:15]

'CNYHEGGGARVAHIM'

In [286]:
DQA1_seq_dict['DQA1*01:05']

{'HVASCGVNLYQFYGPSGQYTHEFDGDEEFYVDLERKETAWRWPEFSKFGGFDPQGALRNMAVAKHNLNIMIKRYN'}

In [289]:
# the first position from alpha is off
# need to be modified to move 1 position before to make DRA, DPA and DQA match with  "../data/pseudosequence_2016_all_X.dat"
"".join(['HVASCGVNLYQFYGPSGQYTHEFDGDEEFYVDLERKETAWRWPEFSKFGGFDPQGALRNMAVAKHNLNIMIKRYN'[ind-4] for ind in pos_alpha])

'GNYHEGGGARVAHIM'

In [337]:
Counter([len(value) for value in list(DQA1_seq_dict.values())])

Counter({1: 34, 2: 1})

In [339]:
[value for value in list(DQA1_seq_dict.values()) if len(value) > 1]

[{'HVASYGVNLYQSYGPSGQFTHEFDGDEQFYVDLGRKETVWCLPV-LRQFRFDPQFALTNIAVTKHNLNILIKRSN',
  'XXXXXGVNLYQSYGPSGQFTHEFDGDEQFYVDLGRKETVWCLPV-LRQFRFDPQFALTNIAVTKHNLNILIKRSN'}]

try finding alignment of DQB, based on 

    DQB_prot.alfas

and 

    ../data/pseudosequence_2016_all_X.dat"

In [290]:
DQB1_seq_dict['DQB1*05:01']

{'FVYQFKGLCYFTNGTERVRGVTRHIYNREEYVRFDSDVGVYRAVTPQGRPVAEYWNSQKEVLEGARASVDRVCRHNYEVAYRGIL'}

In [294]:
HLA_2_pseudo_dict['HLA-DQA10105-DQB10501'][15:]

'YFGGTHYVVGASRVHVAGI'

In [295]:
"".join(['FVYQFKGLCYFTNGTERVRGVTRHIYNREEYVRFDSDVGVYRAVTPQGRPVAEYWNSQKEVLEGARASVDRVCRHNYEVAYRGIL'[ind-7] for ind in pos_beta]) == 'YFGGTHYVVGASRVHVAGI'

True

In [291]:
Counter([len(value) for value in list(DQB1_seq_dict.values())])

Counter({1: 743, 2: 5})

In [292]:
[value for value in list(DQB1_seq_dict.values()) if len(value) > 1]

[{'FVYQFKGLCYFTNGTERVRGVTRHIYNREEYVRFDSDVGVYRAVTPQGRPSAEYWNSQKEVLEGARASVDRVCRHNYEVAYRGIL',
  'XXYQFKGLCYFTNGTERVRGVTRHIYNREEYVRFDSDVGVYRAVTPQGRPSAEYWNSQKEVLEGARASVDRVCRHNYEVAYRGIL'},
 {'FVYQFKGMCYFTNGTERVRLVTRHIYNREEYARFDSDVGVYRAVTPQGRPDAEYWNSQKEVLERTRAELDTVCRHNYEVGYRGIL',
  'XXYQFKGMCYFTNGTERVRLVTRHIYNREEYARFDSDVGVYRAVTPQGRPDAEYWNSQKEVLERTRAELDTVCRHNYEVGYRGIX'},
 {'FVFQFKGMCYFTNGTERVRLVTRYIYNREEYARFDSDVGVYRAVTPQGRPVAEYWNSQKEVLEGTRAELDTVCRHNYEVAFRGIL',
  'FVFQFKGMCYFTNGTERVRLVTRYIYNREEYARFDSDVGVYRAVTPQGRPVAEYWNSQKEVLEGTRAELDTVCRHNYEVAFRGXX'},
 {'FVYQFKGMCYFTNGTERVRLVTRYIYNREEYARFDSDVGVYRAVTPQGRPVAEYWNSQKEVLERTRAELDTVCRHNYEVAFRGIL',
  'XXYQFKGMCYFTNGTERVRLVTRYIYNREEYARFDSDVGVYRAVTPQGRPVAEYWNSQKEVLERTRAELDTVCRHNYEVAFRGIL'},
 {'FVYQFKAMCYFTNGTERVRYVTRYIYNREEYARFDSDVEVYRAVTPLGPPDAEYWNSQKEVLERTRAELDTVCRHNYQLELRTTL',
  'XXYQFKAMCYFTNGTERVRYVTRYIYNREEYARFDSDVEVYRAVTPLGPPDAEYWNSQKEVLERTRAELDTVCRHNYQLELRTTL'}]

In [296]:
"".join(['FVYQFKGLCYFTNGTERVRGVTRHIYNREEYVRFDSDVGVYRAVTPQGRPSAEYWNSQKEVLEGARASVDRVCRHNYEVAYRGIL'[ind-7] for ind in pos_beta])

'YFGGTHYSVGASRVHVAGI'

In [298]:
"".join(['XXYQFKGLCYFTNGTERVRGVTRHIYNREEYVRFDSDVGVYRAVTPQGRPSAEYWNSQKEVLEGARASVDRVCRHNYEVAYRGIL'[ind-7] for ind in pos_beta]) == \
"".join(['FVYQFKGLCYFTNGTERVRGVTRHIYNREEYVRFDSDVGVYRAVTPQGRPSAEYWNSQKEVLEGARASVDRVCRHNYEVAYRGIL'[ind-7] for ind in pos_beta])

True

In [436]:
# this part finally check the consistency between the pseudo sequences and the prot.alfas files. 

# materials:

# this first dictionary holds the corresponding two alleles of each HLA-II pair
# sep_hla_II_dict
# this second dictionary holds the translate of one HLA-II pair to the pseudo sequence in "../data/pseudosequence_2016_all_X.dat"
# trans_hla_II_dict

# the sequence below gives the unqiue DRA sequence:
# 'HVIIQ-AEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFGRFASFEAQGALANIAVDKANLEIMTKRSN'
# the following dictionaries provide DRB, DPA, DPB, DQA, DQB sequences:
# DRB_seq_dict
# DPA1_seq_dict
# DPB1_seq_dict
# DQA1_seq_dict
# DQB1_seq_dict 

# this dictionary below provides the pseudo sequence for HLA-II pairs
# HLA_2_pseudo_dict

# 15 positions on alpha chain (off by 4), 
# move the starting position 1 to the left
# if multiple choice and one with an "X", use the other
# pos_alpha
# 19 positions on beta chain (off by 7)



def get_a_half(lookup_dict, allele, adjust_pos_alpha):
    seq_candids = list(lookup_dict[allele])
    seq_pseudos = list(set(["".join([item[ind-4] for ind in adjust_pos_alpha]) for item in seq_candids]))
    seq_pseudo_noX = [seq for seq in seq_pseudos if "X" not in seq]
    if len(seq_pseudo_noX) == 1:
        return True, seq_pseudo_noX[0]
    else:
        return False, ""
    
def get_b_half(lookup_dict, allele, pos_beta):
    seq_candids = list(lookup_dict[allele])
    seq_pseudos = list(set(["".join([item[ind-7] for ind in pos_beta]) for item in seq_candids]))
    seq_pseudo_noX = [seq for seq in seq_pseudos if "X" not in seq]
    if len(seq_pseudo_noX) == 1:
        return True, seq_pseudo_noX[0]
    else:
        return False, ""



match_flags = []

adjust_pos_alpha = [pos for pos in pos_alpha]
adjust_pos_alpha[0] = adjust_pos_alpha[0] - 1



# separate the HLA-II pairs into two alleles each
# translate them into the names in file "../data/pseudosequence_2016_all_X.dat"
for item in list_HLA_II_complete:
    if item[:8] == "HLA-DQAB":
        item_1, item_2 = sep_hla_II_dict[item]
        sub_flag_1, seq1 = get_a_half(DQA1_seq_dict, item_1, adjust_pos_alpha)
        sub_flag_2, seq2 = get_b_half(DQB1_seq_dict, item_2, pos_beta)
        if (not sub_flag_1) or (not sub_flag_2):
            print("seq_pseudo_noX len is not 1")
            print("item = ", item)
            break
        seq_pseudo = HLA_2_pseudo_dict[trans_hla_II_dict[item]]
        match_flags += [(seq1 + seq2) == seq_pseudo]        
    elif item[:8] == "HLA-DPAB":
        item_1, item_2 = sep_hla_II_dict[item]
        sub_flag_1, seq1 = get_a_half(DPA1_seq_dict, item_1, adjust_pos_alpha)
        sub_flag_2, seq2 = get_b_half(DPB1_seq_dict, item_2, pos_beta)
        if (not sub_flag_1) or (not sub_flag_2):
            print("seq_pseudo_noX len is not 1")
            print("item = ", item)
            break
        seq_pseudo = HLA_2_pseudo_dict[trans_hla_II_dict[item]]
        match_flags += [(seq1 + seq2) == seq_pseudo]    
    elif item[:8] == "HLA-DRB1":
        item_1, item_2 = sep_hla_II_dict[item]
        seq1 = 'QEFFIASGAAVDAIM'
        sub_flag, seq2 = get_b_half(DRB_seq_dict, item_2, pos_beta)
        if not sub_flag:
            print("seq_pseudo_noX len is not 1")
            print("item = ", item)
            break
        seq_pseudo = HLA_2_pseudo_dict[trans_hla_II_dict[item]]
        match_flags += [(seq1 + seq2) == seq_pseudo]
    else:
        print("error found, first eight letters exception")
        print(item)
        break

In [471]:
sum(match_flags)/len(match_flags)

0.7481481481481481

Looking into one specific allele DQA1*05:01, the sequence from DQA_prot.alfas is:

'HVASYGVNLYQSYGPSGQYTHEFDGDEQFYVDLGRKETVWCLPV-LRQFRFDPQFALTNIAVLKHNLNSLIKRSN'

and the one from NetMHC_II_pan-3.0 paper is:

'DLGRKETVWCLPVLRQFR-FDPQFALTNIAVLKHNLNSLIKRSNSTAATN----------'

My understanding is that it is due to different ways of aligning the sequences. The deletion on position 53 from NetMHC_II_pan-3.0 paper is not considered as a deletion in DQA_prot.alfas, but a delection is considered by DQA_prot.alfas before position 48 (using the index from NetMHC_II_pan-3.0). Thus due to that pos_alpha does not contain other positions in the area, the reconstructed pseudo sequences and those from 

    ../../data/intermediate_data/DeWitt_2018/pseudosequence_2016_all_X.dat

still match on other positions except on 52 & 53 (1-indexed under the setting of NetMHC_II_pan-3.0).


In [442]:
list_HLA_II_complete[1]
HLA_2_pseudo_dict[trans_hla_II_dict['HLA-DQAB*05:01_06:02']]
item_1, item_2 = sep_hla_II_dict['HLA-DQAB*05:01_06:02']
item_1
item_2

sub_flag_1, seq1 = get_a_half(DQA1_seq_dict, item_1, adjust_pos_alpha)
sub_flag_2, seq2 = get_b_half(DQB1_seq_dict, item_2, pos_beta)

seq1
seq2

sub_flag_2

In [516]:
list(DQA1_seq_dict['DQA1*05:01'])[0][48:]

'FRFDPQFALTNIAVLKHNLNSLIKRSN'

In [452]:
seq1 + seq2

'YNYHQFRFATVLHSLFFGLTYYDVGTETVHVAGI'

In [517]:
HLA_2_pseudo_dict[trans_hla_II_dict['HLA-DQAB*05:01_06:02']]

'YNYHQRXFATVLHSLFFGLTYYDVGTETVHVAGI'

In [454]:
seq_candids = list(DQA1_seq_dict[item_1])
seq_pseudos = list(set(["".join([item[ind-4] for ind in adjust_pos_alpha]) for item in seq_candids]))
seq_candids[0][30:36]

In [467]:
pos_alpha

[9, 11, 22, 24, 31, 52, 53, 58, 59, 61, 65, 66, 68, 72, 73]

In [455]:
seq_pseudos

['YNYHQFRFATVLHSL']

In [458]:
list_HLA_II_complete[3]

'HLA-DQAB*05:05_06:04'

In [459]:
HLA_2_pseudo_dict[trans_hla_II_dict['HLA-DQAB*05:05_06:04']]

'YNYHQRXFATVLHSLYFGLTHYVVRTETVHVGGI'

In [460]:
item_1, item_2 = sep_hla_II_dict['HLA-DQAB*05:05_06:04']

In [462]:
item_2

'DQB1*06:04'

In [ ]:
sub_flag_1, seq1 = get_a_half(DQA1_seq_dict, item_1, adjust_pos_alpha)
sub_flag_2, seq2 = get_b_half(DQB1_seq_dict, item_2, pos_beta)

In [509]:
#[list_HLA_II_complete[i] for i in range(135) if not match_flags[i]]

In [510]:
#[list_HLA_II_complete[i] for i in range(135) if match_flags[i]]

In [474]:
item = 'HLA-DPAB*01:03_17:01'

In [484]:
HLA_2_pseudo_dict[trans_hla_II_dict[item]][:15] == 'YAFFMFSGGAILNTL'

True

In [476]:
item_1, item_2 = sep_hla_II_dict[item]

In [491]:
item_2

'DPB1*17:01'

In [480]:
DPA1_seq_dict['DPA1*01:03']

{'HVSTY-AAFVQTHRPTGEFMFEFDEDEMFYVDLDKKETVWHLEEFGQAFSFEAQGGLANIAILNNNLNTLIQRSN',
 'XXXXX-XAFVQTHRPTGEFMFEFDEDEMFYVDLDKKETVWHLEEFGQAFSFEAQGGLANIAILNNNLNTLIQRSN'}

In [481]:
seq_candids = list(DPA1_seq_dict[item_1])
seq_pseudos = list(set(["".join([item[ind-4] for ind in adjust_pos_alpha]) for item in seq_candids]))

In [485]:
seq_pseudos

['YAFFMFSGGAILNTL', 'XAFFMFSGGAILNTL']

In [486]:
seq_candids = list(DPB1_seq_dict[item_2])
seq_pseudos = list(set(["".join([item[ind-7] for ind in pos_beta]) for item in seq_candids]))

In [498]:
seq_pseudos[0]

'HLQFEYFDIEEVRMHLDVT'

In [497]:
HLA_2_pseudo_dict[trans_hla_II_dict[item]][15:]

'YLQFEYFDIEEVRMHLDVT'

In [489]:
seq_candids

['YVHQLRQECYAFN--GTQRFLERYIYNREEFVRFDSDVGEFRAVTELGRPDEDYWNSQKDILEEERAVPDRMCRHNYELDEAVTL']

In [490]:
pos_beta

[9, 11, 13, 26, 28, 30, 47, 57, 67, 70, 71, 74, 77, 78, 81, 85, 86, 89, 90]

In [499]:
item_2 = 'DPB1*10:01'

In [500]:
seq_candids = list(DPB1_seq_dict[item_2])
seq_pseudos = list(set(["".join([item[ind-7] for ind in pos_beta]) for item in seq_candids]))

In [501]:
seq_pseudos

['HLQFEYFDIEEVRVHLDVT']

In [502]:
seq_candids 

['YVHQLRQECYAFN--GTQRFLERYIYNREEFVRFDSDVGEFRAVTELGRPDEEYWNSQKDILEEERAVPDRVCRHNYELDEAVTL']

In [503]:
HLA_2_pseudo_dict[trans_hla_II_dict['HLA-DPAB*02:01_10:01']][15:]

'YLQFEYFDIEEVRVHLDVT'

In [505]:
'YLQFEYFDIEEVRMHLDVT'
'YLQFEYFDIEEVRVHLDVT'

False

In [504]:
'YVHQLRQECYAFN--GTQRFLERYIYNREEFVRFDSDVGEFRAVTELGRPDEDYWNSQKDILEEERAVPDRMCRHNYELDEAVTL' == \
'YVHQLRQECYAFN--GTQRFLERYIYNREEFVRFDSDVGEFRAVTELGRPDEEYWNSQKDILEEERAVPDRVCRHNYELDEAVTL'

False

The way to fix the verification procedure for now:
    
(0) If DQA falls into the list extra_adjust_DQAs, need to modify positions 52 & 53 (1-indexed under NetMHC_II_pan-3.0);

(1) If DPB falls into the list extra_adjust_DPBs, need to manually modify the first position to 'Y'.

In [538]:
extra_modify_DQAs = ['DQA1*05:04', 'DQA1*05:07', 'DQA1*05:01', 'DQA1*05:06', 'DQA1*05:03', \
                    'DQA1*05:02', 'DQA1*05:08', 'DQA1*05:05', 'DQA1*05:11', 'DQA1*05:10', \
                    'DQA1*05:09', 'DQA1*04:02', 'DQA1*04:04', 'DQA1*06:01', 'DQA1*06:02', \
                    'DQA1*02:01', 'DQA1*04:01']

extra_modify_DPBs = ['DPB1*17:01', 'DPB1*10:01']

In [539]:
# this part finally check the consistency between the pseudo sequences and the prot.alfas files, with additional
# adjustments

# materials:

# this first dictionary holds the corresponding two alleles of each HLA-II pair
# sep_hla_II_dict
# this second dictionary holds the translate of one HLA-II pair to the pseudo sequence in "../data/pseudosequence_2016_all_X.dat"
# trans_hla_II_dict

# the sequence below gives the unqiue DRA sequence:
# 'HVIIQ-AEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFGRFASFEAQGALANIAVDKANLEIMTKRSN'
# the following dictionaries provide DRB, DPA, DPB, DQA, DQB sequences:
# DRB_seq_dict
# DPA1_seq_dict
# DPB1_seq_dict
# DQA1_seq_dict
# DQB1_seq_dict 

# the lists of DQAs and DPBs that need additional adjustment when getting pseudo sequences
# extra_modify_DQAs
# extra_modify_DPBs

# this dictionary below provides the pseudo sequence for HLA-II pairs
# HLA_2_pseudo_dict

# 15 positions on alpha chain (off by 4), 
# move the starting position 1 to the left
# if multiple choice and one STARTS WITH (different from the previous contains "X" condition) an "X", use the other
# pos_alpha
# 19 positions on beta chain (off by 7)


def get_a_half_modify(lookup_dict, allele, adjust_pos_alpha, modify_flag):
    seq_candids = list(lookup_dict[allele])
    if modify_flag:
        modify_adjust_pos_alpha = [pos for pos in adjust_pos_alpha]
        modify_adjust_pos_alpha[5] = 53
        seq_pseudos_modify = list(set(["".join([item[ind-4] for ind in modify_adjust_pos_alpha]) for item in seq_candids]))
        seq_pseudos = [item[:6] + 'X' + item[7:] for item in seq_pseudos_modify]   
    else:
        seq_pseudos = list(set(["".join([item[ind-4] for ind in adjust_pos_alpha]) for item in seq_candids]))
    seq_pseudo_noX = [seq for seq in seq_pseudos if seq[0]!= 'X']
    if len(seq_pseudo_noX) == 1:
        return True, seq_pseudo_noX[0]
    else:
        return False, ""
    
def get_b_half_modify(lookup_dict, allele, pos_beta, modify_flag):
    seq_candids = list(lookup_dict[allele])
    if modify_flag:
        seq_pseudos_modify = list(set(["".join([item[ind-7] for ind in pos_beta]) for item in seq_candids]))
        seq_pseudos = ['Y'+item[1:] for item in seq_pseudos_modify]
    else:
        seq_pseudos = list(set(["".join([item[ind-7] for ind in pos_beta]) for item in seq_candids]))
    seq_pseudo_noX = [seq for seq in seq_pseudos if "X" not in seq]
    if len(seq_pseudo_noX) == 1:
        return True, seq_pseudo_noX[0]
    else:
        return False, ""



modify_match_flags = []

adjust_pos_alpha = [pos for pos in pos_alpha]
adjust_pos_alpha[0] = adjust_pos_alpha[0] - 1



# separate the HLA-II pairs into two alleles each
# translate them into the names in file "../data/pseudosequence_2016_all_X.dat"
for item in list_HLA_II_complete:
    if item[:8] == "HLA-DQAB":
        item_1, item_2 = sep_hla_II_dict[item]
        modify_flag = (item_1 in extra_modify_DQAs)
        sub_flag_1, seq1 = get_a_half_modify(DQA1_seq_dict, item_1, adjust_pos_alpha, modify_flag)
        sub_flag_2, seq2 = get_b_half_modify(DQB1_seq_dict, item_2, pos_beta, False)
        if (not sub_flag_1) or (not sub_flag_2):
            print("seq_pseudo_noX len is not 1")
            print("item = ", item)
            break
        seq_pseudo = HLA_2_pseudo_dict[trans_hla_II_dict[item]]
        modify_match_flags += [(seq1 + seq2) == seq_pseudo]        
    elif item[:8] == "HLA-DPAB":
        item_1, item_2 = sep_hla_II_dict[item]
        modify_flag = (item_2 in extra_modify_DPBs)
        sub_flag_1, seq1 = get_a_half_modify(DPA1_seq_dict, item_1, adjust_pos_alpha, False)
        sub_flag_2, seq2 = get_b_half_modify(DPB1_seq_dict, item_2, pos_beta, modify_flag)
        if (not sub_flag_1) or (not sub_flag_2):
            print("seq_pseudo_noX len is not 1")
            print("item = ", item)
            break
        seq_pseudo = HLA_2_pseudo_dict[trans_hla_II_dict[item]]
        modify_match_flags += [(seq1 + seq2) == seq_pseudo]    
    elif item[:8] == "HLA-DRB1":
        item_1, item_2 = sep_hla_II_dict[item]
        seq1 = 'QEFFIASGAAVDAIM'
        sub_flag, seq2 = get_b_half_modify(DRB_seq_dict, item_2, pos_beta, False)
        if not sub_flag:
            print("seq_pseudo_noX len is not 1")
            print("item = ", item)
            break
        seq_pseudo = HLA_2_pseudo_dict[trans_hla_II_dict[item]]
        modify_match_flags += [(seq1 + seq2) == seq_pseudo]
    else:
        print("error found, first eight letters exception")
        print(item)
        break

In [544]:
sum(modify_match_flags)/len(modify_match_flags)

1.0